# 1D Wave Propagation through a Heterogeneous Domain

In this notebook, we no longer solve domains monolithically (single time step), we introduce the concept of multi time
stepping with an algorithm that calls two instances of the Simple Integrator for two individually solved subdomains

## Multi-Time Step Integration 
This module describes a multi-time step integration algorithm for two subdomains through enforcing
continuity of acceleration across the interface of subdomains

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imageio 
import sys
import os
sys.path.append('..')
from SimpleIntegrator import SimpleIntegrator
from BoundaryConditions import VelBoundaryConditions as vbc
from BoundaryConditions import AccelBoundaryConditions as abc


## Create a Multi Time Step Class

We create a module for Integrating with Multiple Time Steps
It contains the following methods:
 - Calculation of Time Step Ratios
 - Handling of the Integration between both subdomains
 

In [ ]:
class MultiTimeStep:

    """
    Constructor for the subcycling class
    It accepts two domains a large and a small one
    They are both SimpleIntegrators, and they ratio is a non-integer number:
    LARGE     |   SMALL
    *----*----*--*--*--*--*
    """
    def __init__(self, largeDomain: SimpleIntegrator, smallDomain: SimpleIntegrator):
        
        self.large = largeDomain
        self.small = smallDomain
        self.large.mass[-1] += self.small.mass[0] 
        # Time step Ratio Computations
        self.large_tpredicted = 0.0
        self.small_tpredicted = 0.0
        self.nextTimeStepRatio = 0.0 
        self.currTimeStepRatio = 0.0
        # Pullback Values
        self.large_alpha = 0.0
        self.small_alpha = 0.0

    def calc_timestep_ratios(self):
        self.small_tpredicted = self.small.t + self.small.dt
        self.large_tpredicted = self.large.t + self.large.dt
        self.currTimeStepRatio = (self.small.t - self.large.t) / (self.large_tpredicted - self.large.t)
        self.nextTimeStepRatio = ((self.small.t + self.small.dt) - self.large.t) / (self.large_tpredicted - self.large.t)

    def integrate(self):
    
        if ((self.large.t == 0) and (self.small.t == 0)):
            self.small.assemble_internal()
            self.large.assemble_internal()
            self.large.f_int[-1] += self.small.f_int[0]
            self.calc_timestep_ratios()

        while (self.nextTimeStepRatio <= 1 or (self.currTimeStepRatio <= 1 and self.nextTimeStepRatio <= 1.000001)):
            largeForce = self.large.f_int[-1]
            largeMass = self.large.mass[-1]
            def accelCoupling(): return -largeForce / largeMass

            self.small.a_bc.indexes.append(0)
            self.small.a_bc.accelerations.append(accelCoupling)

            self.small.single_tstep_integrate()
            self.small.assemble_internal()
            self.calc_timestep_ratios()

        # Compute Pullback Values
        self.alpha_L = 1 - ((self.large_tpredicted - self.small.t)/(self.large_tpredicted - self.large.t))
        self.alpha_s = 1 - ((self.small_tpredicted - self.large_tpredicted)/(self.small_tpredicted - self.small.t))

        if (self.alpha_L >= self.alpha_s):
            self.large.dt = self.alpha_L * self.large.dt
        elif (self.alpha_s > self.alpha_L):
            self.small.dt = self.alpha_s * self.small.dt

        self.large.single_tstep_integrate()
        self.large.assemble_internal()        
        self.calc_timestep_ratios()

        self.large.f_int[-1] += self.small.f_int[0]

## Output Plots for Multi Time Step Class

A Class is created to plot the FEM 1D Problems for the Heterogeneous Domain Case
This includes the creation of a plot for:
 - Velocity throughout the domain (.gif)